In [1]:
import tensorflow as tf

In [2]:
def pretty_print_guess_tensor(const_guess, operand_guess, operator_guess):
    # TODO: const_guess

    s = []

    for t in operand_guess:
        s += [f'x_{tf.argmax(t)}']

    operator_lookup = ['+','-', '*','/']
    result = s[::]
    for i, op_one_hot in enumerate(operator_guess):
        operators = tf.argmax(op_one_hot,axis=-1)
        left = result[::2]
        right = (result[1:] + result[:1])[::2]
        ops = operators[:len(left)]
        result = []
        for l, op, r in zip(left, ops, right):
            result += [f'({l} {operator_lookup[op]} {r})']


    return ' '.join(result)

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)

pretty_print_guess_tensor(const_guess, operand_guess, operator_guess)

'(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))'

In [3]:
@tf.function
def dot(x, y):
    r = tf.multiply(x, y)
    return tf.reduce_sum(r, -1)

x = tf.constant([
    [2,2,2],
    [3,3,3]
])

dot(x, x)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([12, 27])>

In [4]:
@tf.function
def resolve_values(const_guess, values, operand_guess):
    # TODO: const_guess

    operand_count = tf.shape(operand_guess)[0]
    values = tf.expand_dims(values, axis=0)
    values = tf.tile(values, [operand_count,1])
    
    result = dot(values, operand_guess)

    return result

v1 = tf.range(NUM_LEAVES)
cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
values = tf.cast(v1,dtype=tf.float32)

target = tf.constant([1,1,1,1,1,1,1,1],dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(operand_guess)
    result = resolve_values(const_guess, values, operand_guess)
    loss = tf.nn.l2_loss(target - result)
tf.print(result)
tf.print(loss)
tf.print(tape.gradient(loss, operand_guess))

[0 1 2 ... 5 6 7]
46
[[0 -1 -2 ... -5 -6 -7]
 [0 0 0 ... 0 0 0]
 [0 1 2 ... 5 6 7]
 ...
 [0 4 8 ... 20 24 28]
 [0 5 10 ... 25 30 35]
 [0 6 12 ... 30 36 42]]


In [5]:
@tf.function
def operate(operands, operators):
    left = operands[::2]
    right = tf.roll(operands, shift=-1, axis=0)[::2]

    r_add = left + right
    r_sub = left - right
    r_mul = left * right
    r_div = tf.math.divide_no_nan(left, right)

    r = tf.stack([r_add, r_sub, r_mul, r_div], axis=1)

    return dot(r, operators)

operands = tf.range(NUM_LEAVES, dtype=tf.float32)
v2 = tf.range(NUM_OPERATORS)
operators = tf.constant([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1],
],dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(operands)
    tape.watch(operators)
    result = operate(operands, operators)

tf.print(result)
tf.print(tf.reshape(tape.gradient(result, operands),(2,4)))
tf.print(tape.gradient(result, operators))

[1 -1 20 0.857142866]
[[1 1 1 -1]
 [5 4 0.142857149 -0.122448981]]
[[1 -1 0 0]
 [5 -1 6 0.666666687]
 [9 -1 20 0.8]
 [13 -1 42 0.857142866]]


In [6]:
def eager_process_block(operands, operators_arr):
    acc = operands

    for operators in operators_arr:
        num_operands = tf.shape(acc)[0]
        operators = operators[:num_operands // 2]
        acc = operate(acc, operators)

    return acc

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)
operands = resolve_values(const_guess, values, operand_guess)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(operands)
    tape.watch(operator_guess)
    result = eager_process_block(operands, operator_guess)

tf.print(pretty_print_guess_tensor(const_guess, operand_guess, operator_guess))
x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7 = list(range(8))
tf.print((((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7))))
tf.print(result)
tf.print(tf.reshape(tape.gradient(result, operands),(2,4)))
tf.print(tape.gradient(result, operator_guess))

(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
19.142857142857142
[19.1428566]
[[1 1 1 -1]
 [5 4 -0.142857149 0.122448981]]
[[[1 -1 0 0]
  [5 -1 6 0.666666687]
  [9 -1 20 0.8]
  [-13 1 -42 -0.857142866]]

 [[0 2 -1 -1]
  [20.8571434 19.1428566 17.1428566 23.333334]
  [0 0 0 0]
  [0 0 0 0]]

 [[19.1428566 -19.1428566 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


In [7]:
@tf.function
def unrolled_process_block_3(operands, operators_arr):
    acc = operands

    # Level 1
    operators = operators_arr[0]
    operators = operators[:4]
    acc = operate(acc, operators)

    # Level 2
    operators = operators_arr[1]
    operators = operators[:2]
    acc = operate(acc, operators)

    # Level 3
    operators = operators_arr[2]
    operators = operators[:1]
    acc = operate(acc, operators)

    return acc

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)
operands = resolve_values(const_guess, values, operand_guess)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(operands)
    tape.watch(operator_guess)
    result = unrolled_process_block_3(operands, operator_guess)

tf.print(pretty_print_guess_tensor(const_guess, operand_guess, operator_guess))
x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7 = list(range(8))
tf.print((((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7))))
tf.print(result)
tf.print(tf.reshape(tape.gradient(result, operands),(2,4)))
tf.print(tape.gradient(result, operator_guess))

(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
19.142857142857142
[19.1428566]
[[1 1 1 -1]
 [5 4 -0.142857149 0.122448981]]
[[[1 -1 0 0]
  [5 -1 6 0.666666687]
  [9 -1 20 0.8]
  [-13 1 -42 -0.857142866]]

 [[0 2 -1 -1]
  [20.8571434 19.1428566 17.1428566 23.333334]
  [0 0 0 0]
  [0 0 0 0]]

 [[19.1428566 -19.1428566 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


In [8]:
@tf.function
def to_prob_dist_all(v):
    v2 = tf.sqrt(tf.square(v)+1e-9)
    # v2 = tf.sqrt(tf.square(v))
    m = tf.expand_dims(tf.reduce_sum(v2, axis=-1),-1)
    n = tf.math.divide_no_nan(v2, m)
    return n

tf.print(tf.argmax(operator_guess))
tf.print(tf.argmax(to_prob_dist_all(operator_guess)))

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [9]:
def print_collapsed_result(values, const_guess, operand_guess, operator_guess):
    # TODO: const_guess

    operands = tf.round(operand_guess)
    acc = resolve_values(const_guess, values, operands)
    operators = tf.round(operator_guess)

    result = eager_process_block(acc, operators)

    return result

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)

print_collapsed_result(values, const_guess, operand_guess, operator_guess)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([19.142857], dtype=float32)>

In [10]:
def bind_opt_train_step(opt):
    @tf.function
    def train_step(const_guess, operand_guess, operator_guess, values, target):
        with tf.GradientTape() as tape:
            cg, opg, otg = const_guess, operand_guess, operator_guess
            # cg = tf.nn.softmax(cg)
            # opg = tf.nn.softmax(opg)
            # otg = tf.nn.softmax(otg)
            cg = to_prob_dist_all(cg)
            opg = to_prob_dist_all(opg)
            otg = to_prob_dist_all(otg)
            operands = resolve_values(cg, values, opg)
            result = unrolled_process_block_3(operands, otg)
            loss = tf.nn.l2_loss(result[0] - target)

        variables = [const_guess, operand_guess, operator_guess]
        grads = tape.gradient(loss, variables)
        opt.apply_gradients(zip(grads, variables))

        const_guess.assign(to_prob_dist_all(const_guess))
        operand_guess.assign(to_prob_dist_all(operand_guess))
        operator_guess.assign(to_prob_dist_all(operator_guess))

        return loss

    return train_step

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)

const_guess = tf.Variable(const_guess)
operand_guess = tf.Variable(operand_guess)
operator_guess = tf.Variable(operator_guess)

target = 19.0

opt = tf.keras.optimizers.Adam(3e-4)
# opt = tf.keras.optimizers.SGD(1e-1)
train_step = bind_opt_train_step(opt)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(const_guess)
    tape.watch(operand_guess)
    tape.watch(operator_guess)

    result = train_step(const_guess, operand_guess, operator_guess, values, target)

tf.print(result)
tf.print(tape.gradient(result, operand_guess))
tf.print(tape.gradient(result, operator_guess))

0.00941969
[[-0.000121529039 0 0 ... 0 0 0]
 [0 -8.67843628e-05 0 ... 0 0 0]
 [0 0 -5.21242619e-05 ... 0 0 0]
 ...
 [0 0 0 ... 0.000207901 0 0]
 [0 0 0 ... 0 -1.24201179e-05 0]
 [0 0 0 ... 0 0 1.48564577e-05]]
[[[1.73598528e-05 0 0 0]
  [0 -6.36875629e-05 0 0]
  [0 0 0.000222206116 0]
  [0 0 0 0.000223122537]]

 [[3.66417225e-08 0 0 0]
  [0 -1.66893e-05 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0.000332117081 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


In [11]:
opt = tf.keras.optimizers.Adam(3e-4)
# opt = tf.keras.optimizers.Adam(1e-1)
# opt = tf.keras.optimizers.SGD(1e-1)
train_step = bind_opt_train_step(opt)

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)

const_guess = tf.Variable(const_guess)
operand_guess = tf.Variable(operand_guess)
operator_guess = tf.Variable(operator_guess)

target = 7.0

for i in range(1000):
    loss = train_step(const_guess, operand_guess, operator_guess, values, target)

    if i % 100 == 0:
        cg = const_guess.numpy()
        opg = operand_guess.numpy()
        otg = operator_guess.numpy()
        collapsed_result = print_collapsed_result(values, cg, opg, otg)

        tf.print(i, loss, collapsed_result)
        tf.print(pretty_print_guess_tensor(cg, opg, otg))

0 73.6565 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
100 13.4621687 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
200 2.0916543 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
300 0.278446794 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
400 0.0286345873 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
500 0.00234044786 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
600 0.000269231445 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
700 7.45407378e-05 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
800 3.39467078e-05 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
900 1.92249e-05 [19.1428566]
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
